<a href="https://colab.research.google.com/github/diegopalencia-research/english-verbs-nlp/blob/main/notebooks/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>